In [46]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from scipy.optimize import minimize

import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import sys

In [47]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [48]:
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
sys.path.append('../../../../nadavp/opt_lev_analysis/lib/')
import BeadDataFile
from discharge_tools import *

In [49]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [50]:
import bead_util as bu
# import calib_util as cal
import transfer_func_util as tf
import configuration as config

### Discharge data

In [51]:
step_cal_dir = r'/data/old_trap/20211001/bead1/discharge/recharge_20211013/'
max_file = 1

In [52]:
step_cal_files = []
for root, dirnames, filenames in os.walk(step_cal_dir):
    for filename in fnmatch.filter(filenames, '*' + config.extensions['data']):
        if '_fpga.h5' in filename:
            continue
        step_cal_files.append(os.path.join(root, filename))
step_cal_files.sort(key = bu.find_str)
num_calib_files = len(step_cal_files)
print(num_calib_files)

# Do the step calibration

step_file_objs = []
for filname in step_cal_files[:]:
    try:
        df = bu.DataFile();
        df.load(filname);
        df.load_other_data()
        step_file_objs.append(df);
    except:
        print('***')

175


In [53]:
scaleY = 97.9992158717
scaleZ = scaleY*(418+627)/(392+406)
scaleX = scaleY*(421+368)/(392+406)
scaleX, scaleY, scaleZ

(96.89396155735751, 97.9992158717, 128.33230649865476)

In [54]:
print("Scales are ", scaleY)
bb = step_file_objs[-1]
drive_freq = 139
bandwidth=2
fsamp = 5000
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[5])
voltage = np.std(xx2)*np.sqrt(2)*100
print("voltage: ", voltage)

Scales are  97.9992158717
voltage:  24.0265126779


In [55]:
i = 0
fname = r'/data/old_trap/20211001/bead1/neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_4sec_with_height_fb_redo5/Z_200Vpp_'+str(i)+'.h5'
bb = bu.DataFile();
bb.load(fname);
bb.load_other_data()
print("voltage: ", voltage, np.mean(bb.other_data[1])*100, np.mean(bb.other_data[2])*100)
drive_freq = 139
bandwidth=2
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[1])
voltage_200_1 = np.std(xx2)*np.sqrt(2)*100
xx3 = signal.filtfilt(b, a, bb.other_data[2])
voltage_200_2 = np.std(xx3)*np.sqrt(2)*100
print("voltage: ", voltage_200_1, voltage_200_2)

voltage:  24.0265126779 -0.592538948653 -0.574958113985
voltage:  190.87201524 0.674330527298


In [56]:
etha = 1/scaleZ*327*1.6e-19*voltage
to_epsilon = 1/(2*327*1.6e-19*voltage_200_1*(418)/(418+627))

### Look at neutrality data

In [57]:
amps_old_trap = np.load("all_200Vpp_ol_trap_alternate.npy")
amps_old_trap_sideband = np.load("all_200Vpp_ol_trap_alternate_sideband.npy")

In [120]:
factor = 0.645
factor2 = 0.666

data = amps_old_trap[::2,0]
data2 = amps_old_trap[1::2,0]
data3 = data-factor*data2

data_noise = amps_old_trap_sideband[::2,0]
data2_noise = amps_old_trap_sideband[1::2,0]
data3_noise = data_noise-factor*data2_noise

data_1 = np.abs(amps_old_trap[::2,1])
data2_1 = np.abs(amps_old_trap[1::2,1])
data3_1 = data_1+factor*factor*data2_1

data_1_noise = np.abs(amps_old_trap_sideband[::2,1])

fig, ax= plt.subplots(2,1, figsize=(9.5,7), sharex=True, sharey=True)
ax[0].plot(np.arange(len(data))*2*10/3600, data/1e-16, 'o', mfc='none',  label='200Vpp-z-plus, xy-spin-50Vpp-103kHz')
ax[0].plot(np.arange(len(data))*2*10/3600, data2/1e-16, '+', label='200Vpp-z-minus, xy-spin-50Vpp-103kHz', alpha=0.6)

ax[1].scatter(np.arange(len(data3))*2*10/3600, data3/1e-16, label='200Vpp-z, A parameter', alpha=0.4)

ax[0].plot(np.arange(len(data3))*2*10/3600, np.arange(len(data3))*0, '--k')
ax[1].plot(np.arange(len(data3))*2*10/3600, np.arange(len(data3))*0, '--k')
ax[0].set(xlim=(10,20), ylim=(-3,3),  ylabel=r'Measured response [10$^{-16}$N]')
ax[1].set(xlim=(10,20),  xlabel='Time [hr.]', ylabel=r'A parameter [10$^{-16}$N]')
# ax.legend()
# ax[1].set_xticklabels([])
plt.subplots_adjust(wspace=0, hspace=0.05)

# fig.savefig('fig1_force_vs_time.pdf')
print('1st harmonic, electrode 1: mean, std: ', np.mean(data), np.std(data)/np.sqrt(len(data)), np.std(data_noise)*np.sqrt(10))
print('1st harmonic, electrode 2: mean, std: ', np.mean(data2), np.std(data2)/np.sqrt(len(data2)))
print('1st harmonic, A parameter: mean, std: ', np.mean(data3), np.std(data3)/np.sqrt(len(data3)))
print('1st harmonic, A parameter[epsilon]: mean, std: ', np.mean(data3)*to_epsilon, to_epsilon*np.std(data3)/np.sqrt(len(data3)))

print()
print('2nd harmonic, electrode 1: mean, std: ', np.mean(data_1), np.std(data_1)/np.sqrt(len(data_1)))
print('2nd harmonic, electrode 2: mean, std: ', np.mean(data2_1), np.std(data2_1)/np.sqrt(len(data2_1)))
print('2nd harmonic, B parameter: mean, std: ', np.mean(data3_1), np.std(data3_1)/np.sqrt(len(data3_1)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1st harmonic, electrode 1: mean, std:  -2.17707741095e-17 6.36621564849e-19 1.29052834493e-16
1st harmonic, electrode 2: mean, std:  -3.40064001427e-17 8.26668817635e-19
1st harmonic, A parameter: mean, std:  1.63353982548e-19 4.08094746576e-19
1st harmonic, A parameter[epsilon]: mean, std:  2.04470070078e-05 5.10811919791e-05

2nd harmonic, electrode 1: mean, std:  4.11045816737e-16 1.62547535243e-19
2nd harmonic, electrode 2: mean, std:  8.34111897036e-16 2.84524342932e-19
2nd harmonic, B parameter: mean, std:  7.58057218701e-16 2.0888297375e-19


In [59]:
N = len(data)
data_shifted = np.array([(data[i]+data[i-1])/2 for i in range(1,N)])
data2_shifted = data2[:-1]
data3_shifted = data_shifted-factor*data2_shifted

data_shifted_noise = np.array([(data_noise[i]+data_noise[i-1])/2 for i in range(1,N)])
data2_shifted_noise = data2_noise[:-1]
data3_shifted_noise = data_shifted_noise-factor*data2_shifted_noise

_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(np.arange(len(data3))*2*10/3600, data3, label='200Vpp-z-plus, A parameter')
ax.scatter(np.arange(len(data3_shifted))*2*10/3600, data3_shifted, label='200Vpp-z, corrected A parameter', alpha=0.6)

ax.plot(np.arange(len(data3))*2*10/3600, np.arange(len(data3))*0, '--k')
ax.set(xlabel='Integration time [hr.]', ylabel=r'$\epsilon$')
ax.legend()
print('1st harmonic, A parameter: mean, std: ', np.mean(data3), np.std(data3)/np.sqrt(len(data3)))
print('1st harmonic, corrected A: mean, std: ', np.mean(data3_shifted), np.std(data3_shifted)/np.sqrt(len(data3_shifted)))
print('1st harmonic, A parameter[epsilon]: mean, std: ', np.mean(data3_shifted)*to_epsilon, to_epsilon*np.std(data3_shifted)/np.sqrt(len(data3)))
print('1st harmonic, A-noise parameter[epsilon]: mean, std: ', np.mean(data3_shifted_noise)*to_epsilon, to_epsilon*np.std(data3_shifted_noise)/np.sqrt(len(data3_noise)))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1st harmonic, A parameter: mean, std:  1.63353982548e-19 4.08094746576e-19
1st harmonic, corrected A: mean, std:  1.66529226486e-19 3.27480743115e-19
1st harmonic, A parameter[epsilon]: mean, std:  2.08444520779e-05 4.09894081455e-05
1st harmonic, A-noise parameter[epsilon]: mean, std:  4.13964768462e-07 3.93767882452e-05


In [112]:
force_to_edm = 1/(1.2e5*200)/1.6e-19*1e6
print('EDM, electrode 1: mean, std: ', force_to_edm*np.mean(data), force_to_edm*np.std(data))

print('EDM, tick of 1e-16N on electrode 1: mean, std: ', force_to_edm*1e-16)


EDM, electrode 1: mean, std:  -5.66947242434 20.5462037997
EDM, tick of 1e-16N on electrode 1: mean, std:  26.041666666666668


In [60]:
fig_data_array = np.array([data_shifted, data2_shifted, data3_shifted])
np.save('fig1_data_plus_minus_A_20211129', fig_data_array)

In [110]:
b = data_1+data2_1*0.645**2
b_array = np.array([data_1, -data2_1, b])
np.save('old_trap_data_G1_G2_B', b_array)

In [109]:
np.mean(b)/2000

3.7902860935046499e-19

### Fitting

In [21]:
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp

n_bins = 40
range_pram = 0.02
_,ax = plt.subplots()
entries, bins, patches = ax.hist(data3_shifted*to_epsilon, bins=n_bins, range=(-range_pram, range_pram), alpha=0.5, label='inband response')
entries2, bins2, patches2 = ax.hist(data3_shifted_noise*to_epsilon, bins=n_bins, range=(-range_pram, range_pram), alpha=0.3, label='sideband response')
ax.set(xlabel=r'A-parameter [N]')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
## gaussian fit to the histogram

bins_fit =[(bins[i+1]-bins[i])/2+bins[i] for i in range(len(entries))]

titles = ['', '', '']
popt_list = []
for i,ee in enumerate([entries, entries2]):
    x = bins_fit
    y= np.array(ee)
    sigma_bin = np.sqrt(ee)
    
    n = len(x)                          #the number of data
    mean = 0                  #note this correction
    sigma = 0.03       #note this correction

    def gaus(x,a,x0,sigma):
        return a*np.exp(-(x-x0)**2/(2*sigma**2))

    popt,pcov = curve_fit(gaus,x,y,p0=[1,mean,sigma])
    ax.plot(x,gaus(x,*popt),'ro:',label='fit')
    popt_list.append(popt)
    print("mean: ", popt[1], " ,std (on the mean): ", popt[2]/np.sqrt(len(data)), " ,sigma: ", popt[2])
    print("std from the covariance matrix: ", np.sqrt(pcov[1,1]))
    print('\nNeutrality: ', 1/1.26e14*popt[2]/np.sqrt(len(data)))
    print('Factor needed for neutrality best limit: ', 1/1.26e14*popt[2]/np.sqrt(len(data))/1e-21)
    print()

mean:  1.17131763088e-05  ,std (on the mean):  4.0565115737e-05  ,sigma:  0.00502729290911
std from the covariance matrix:  3.90489599586e-05

Neutrality:  3.21945362992e-19
Factor needed for neutrality best limit:  321.945362992

mean:  3.48497097434e-05  ,std (on the mean):  3.95901922424e-05  ,sigma:  0.0049064692437
std from the covariance matrix:  3.52820792183e-05

Neutrality:  3.1420787494e-19
Factor needed for neutrality best limit:  314.20787494



In [23]:
inband_gaus = popt_list[0]
sideband_gaus = popt_list[1]

fig,ax = plt.subplots()
ax.plot(x,gaus(x,*inband_gaus),'r:o',label='inband')
ax.plot(x,gaus(x,*sideband_gaus),'b*--',label='sideband')
ax.legend()
ax.set(xlabel=r'A-parameter [N]')
fig.savefig('fig4_1_limit_charge.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [104]:
print(np.mean(2*data_1/(1.2e5*200)/1.6e-19*1e6))
print(np.mean(2*data2_1/(1.64e5*200)/1.6e-19*1e6))
print(np.mean(2*data_1/(1.64e5*200)/1.6e-19*1e6)/(196*200))
print(np.mean(2*data2_1/(1.2e5*200)/1.6e-19*1e6)/(196*200))

214.086362884
317.878009541
0.00399613668897
0.0110824816252


In [105]:
fig, ax = plt.subplots()
_ = ax.hist(2*data_1/(1.1e5*200)/1.6e-19*1e6, range=(150,375), bins=100, label='top electrode')
_ = ax.hist(2*data2_1/(1.7e5*200)/1.6e-19*1e6, range=(150,375), bins=100, label='bottom electrode')
# _ = ax.hist(data_1_noise, range=(-1e-17,1e-15), bins=100, label='sideband')
# ax.legend()
ax.set(xlabel='$p_{ac}$ [e $\mu$m]')
# fig.savefig('fig3_induced_edm.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, '$p_{ac}$ [e $\\mu$m]')]

In [180]:
p_ac = np.array([2*data_1/(1.2e5*200)/1.6e-19*1e6, 2*data2_1/(1.64e5*200)/1.6e-19*1e6])
p_ac.shape
np.save('fig3_induced_dipole', p_ac)

In [37]:
z_top_mean = []
z_bottom_mean = []
z_top_std = []
z_bottom_std = []

bin_size = 10
bins = len(data)//bin_size
for i in range(bins):
    z_top_mean.append(np.mean(data_shifted[i*bin_size:(i+1)*bin_size]))
    z_bottom_mean.append(np.mean(data2_shifted[i*bin_size:(i+1)*bin_size]))
    z_top_std.append(np.std(data_shifted[i*bin_size:(i+1)*bin_size]))
    z_bottom_std.append(np.std(data2_shifted[i*bin_size:(i+1)*bin_size]))
z_top_mean = np.array(z_top_mean)
z_top_std = np.array(z_top_std)/np.sqrt(bin_size)
z_bottom_mean = np.array(z_bottom_mean)
z_bottom_std = np.array(z_bottom_std)/np.sqrt(bin_size)

In [38]:
_,ax = plt.subplots(figsize=(9.4,4))
time_top_electrode_shifted = np.arange(len(z_top_mean))*10*bin_size/3600*2
ax.errorbar(time_top_electrode_shifted, z_top_mean, yerr=z_top_std, fmt='o')
ax.errorbar(time_top_electrode_shifted, z_bottom_mean, yerr=z_bottom_std, fmt='o')
ax.plot(time_top_electrode_shifted, time_top_electrode_shifted*0, 'k--', zorder=3)
ax.set(xlabel='Time [hr.]', ylabel=r'Measured Force [N]')
# ax.set(xlim=(10,20))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'Time [hr.]'), Text(0, 0.5, 'Measured Force [N]')]

In [39]:
fig2, ax = plt.subplots()
ax.scatter(z_top_mean, z_bottom_mean)
xxx = np.arange(-2.5,3)*1e-16
signal_model = xxx/-0.66
ax.plot(xxx, signal_model,'k--')
ax.set(xlabel='$F_1 [N]$', ylabel='$F_2 [N]$')
# fig2.savefig('fig2_scatter2.pdf', bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, '$F_1 [N]$'), Text(0, 0.5, '$F_2 [N]$')]

In [40]:
thresh = 0.015
data3_eps = data3_shifted*to_epsilon
data3_filt = np.array([dd for dd in data3_eps if dd<thresh if dd>-thresh])
print(len(data3), len(data3_filt))
print((len(data3_filt)-len(data3))/len(data3)*-100)

15359 15305
0.3515853896738069


In [41]:
thresh = 0.015
data3_eps_noise = data3_shifted_noise*to_epsilon
data3_filt_noise = np.array([dd for dd in data3_eps_noise if dd<thresh if dd>-thresh])
print(len(data3_noise), len(data3_filt_noise))
print((len(data3_filt_noise)-len(data3_noise))/len(data3_noise)*-100)

15359 15319
0.26043362198059766


In [42]:
z_combine_means = []
z_combine_std = []

bin_size = 10
bins = len(data3_filt)//bin_size
for i in range(bins):
    z_combine_means.append(np.mean(data3_filt[i*bin_size:(i+1)*bin_size]))
    z_combine_std.append(np.std(data3_filt[i*bin_size:(i+1)*bin_size]))
z_combine_means = np.array(z_combine_means)
z_combine_std = np.array(z_combine_std)/np.sqrt(bin_size)

z_combine_means_noise = []
z_combine_std_noise = []

bins = len(data3_filt_noise)//bin_size
for i in range(bins):
    z_combine_means_noise.append(np.mean(data3_filt_noise[i*bin_size:(i+1)*bin_size]))
    z_combine_std_noise.append(np.std(data3_filt_noise[i*bin_size:(i+1)*bin_size]))
z_combine_means_noise = np.array(z_combine_means_noise)
z_combine_std_noise = np.array(z_combine_std_noise)/np.sqrt(bin_size)

In [43]:
_,ax = plt.subplots(figsize=(9.4,4))
iteration = np.arange(len(z_top_mean))*10*bin_size*2/3600
ax.plot(iteration, iteration*0, 'r--')
A = z_top_mean-z_bottom_mean*factor
A_err = np.sqrt(z_top_std**2+(z_bottom_std*factor)**2)
A_err_noise = np.sqrt(z_top_std**2+(z_bottom_std*factor)**2)
ax.errorbar(iteration, A, yerr=A_err, fmt='o', label='A parameter', alpha=0.3)
ax.legend()
ax.set(xlabel='Measuring time [hr.]', ylabel='Measured force [N]')
# ax.errorbar(iteration, (z_top_means-z_bottom_mean*0.66)*to_epsilon, yerr=z_bottom_std/np.sqrt(10)*to_epsilon, fmt='o')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'Measuring time [hr.]'), Text(0, 0.5, 'Measured force [N]')]

In [44]:
A_epsion = z_combine_means
A_err_epsilon = z_combine_std_noise[:-1]
def calculate_pl(force_):
    chi2 = np.power((A_epsion-force_)/A_err_epsilon, 2)
    return np.sum(chi2)

In [45]:
_,ax = plt.subplots()
pl_range = np.arange(-1e-4,1e-4,2e-6)
pl = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range])
ax.plot(pl_range*100000, pl-np.min(pl), 'o--')
ax.plot(pl_range*100000, pl_range*0+2.7, 'r--')
ax.set(xlabel=r'$\epsilon [10^5]$', ylabel='PL')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, '$\\epsilon [10^5]$'), Text(0, 0.5, 'PL')]

In [35]:
epsilon_array = np.array([data3_shifted, data3_shifted_noise])*to_epsilon

In [103]:
epsilon_array.shape, 15358*10/3600*2

((2, 15358), 85.32222222222222)

In [36]:
np.save("epslion_old_trap", epsilon_array)

In [272]:
_,ax = plt.subplots()
pl_range = np.arange(-2e-4,1e-4,1e-5)
pl = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range])
ax.plot(pl_range*100000, pl-np.min(pl), 'o--', label='-2log(likelihood)')
ax.plot(pl_range*100000, pl_range*0+2.7, 'r--')
ax.set(xlabel=r'$\epsilon [10^5]$', ylabel='PL',)

pl_range_plus = np.arange(0,1e-4,1e-5)
pl_plus = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range_plus])
ax.plot(pl_range_plus*100000, pl_plus-np.min(pl_plus), 'o--', label='profile likelihood')

pl_range_minus = np.arange(-2e-4,pl_range[np.argmin(pl)],1e-5)
pl_minus = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range_minus])
ax.plot(pl_range_minus*100000, pl_minus-np.min(pl_minus), 'o--', label='profile likelihood')
ax.set(xlim=(-15,8), ylim=(-0.1,6))
ax.legend()
print('mean ', pl_range[np.argmin(pl)])
print('limit ', np.interp(2.71, (pl_minus-np.min(pl_minus))[::-1], pl_range_minus[::-1]), np.interp(2.71, pl_plus-np.min(pl_plus), pl_range_plus))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

mean  2e-05
limit  -4.41486558343e-05 8.44962290276e-05


In [ ]:
_,ax = plt.subplots()
pl_range_plus = np.arange(0,1e-4,1e-5)
pl_plus = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range_plus])
ax.plot(pl_range_plus*100000, pl_plus-np.min(pl_plus), 'o--')
ax.plot(pl_range_plus*100000, pl_range_plus*0+2.7, 'r--')

ax.set(xlabel=r'$\epsilon [10^5]$', ylabel='PL')
pl_range[np.argmin(pl)], np.interp(2.71, pl_plus-np.min(pl_plus), pl_range_plus)

In [ ]:
_,ax = plt.subplots()
pl_range_minus = np.arange(-2e-4,pl_range[np.argmin(pl)],1e-5)
pl_minus = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range_minus])
ax.plot(pl_range_minus*100000, pl_minus-np.min(pl_minus), 'o--')
ax.plot(pl_range_minus*100000, pl_range_minus*0+2.7, 'r--')

ax.set(xlabel=r'$\epsilon [10^5]$', ylabel='PL')
pl_range[np.argmin(pl)], np.interp(2.71, (pl_minus-np.min(pl_minus))[::-1], pl_range_minus[::-1])